# Command-line conversion of LCModel basis set
This notebook demonstrates the conversion of an existing LCModel basis set (`.basis` format).

In addition to the basic conversion step, three extra steps are performed:
1. Removal of the TMS peak from all spectra
2. Removal of duplicated and unwanted basis spectra
3. Addition of FSL-MRS default MM peaks

This converted spectrum will subsequently be used to fit the example MRSI data.

## Step 1 - conversion
We use the `basis_tools` command to convert the basis set. The conversion process preserved the phase/frequency convention fot he LCModel basis set. So to match that expected by FSL-MRS we also reverse it.
    

In [3]:
%sx basis_tools convert example_data/example_mrsi/3T_slaser_32vespa_1250.BASIS 3T_slaser_32vespa_1250
%sx basis_tools conj 3T_slaser_32vespa_1250 3T_slaser_32vespa_1250

Traceback (most recent call last):
  File "/home/ppzdc/miniconda3/envs/fsl_mrs_env/bin/basis_tools", line 33, in <module>
    sys.exit(load_entry_point('fsl-mrs', 'console_scripts', 'basis_tools')())
  File "/mnt/d/git/fsl_mrs/fsl_mrs/scripts/basis_tools.py", line 224, in main
    args.func(args)
  File "/mnt/d/git/fsl_mrs/fsl_mrs/scripts/basis_tools.py", line 260, in convert
    from fsl_mrs.utils import basis_tools
  File "/mnt/d/git/fsl_mrs/fsl_mrs/utils/basis_tools.py", line 17, in <module>
    from fsl_mrs.utils.preproc import hlsvd, zero_spectrum
  File "/mnt/d/git/fsl_mrs/fsl_mrs/utils/preproc/__init__.py", line 4, in <module>
    from fsl_mrs.utils.preproc.phasing import phaseCorrect, applyPhase, applyLinPhase
  File "/mnt/d/git/fsl_mrs/fsl_mrs/utils/preproc/phasing.py", line 15, in <module>
    from fsl_mrs.utils.preproc.remove import hlsvd
  File "/mnt/d/git/fsl_mrs/fsl_mrs/utils/preproc/remove.py", line 10, in <module>
    import hlsvdpropy
  File "/home/ppzdc/miniconda3/env

256

Let's view the converted basis set

In [ ]:
from fsl_mrs.utils import mrs_io
import matplotlib.pyplot as plt
basis = mrs_io.read_basis('3T_slaser_32vespa_1250')
fig = plt.figure(figsize=(8, 8))
_ = basis.plot()

## Step 2 - reference peak removal
LCModel basis sets have a reference peak in them at 0.0 ppm. This is removed by default in LCModel, but it isn't expected in FSL-MRS. Even the peak falls outside the default optimisation region, if you leave the reference peak in the summation across all metabolites can grow so large (as everything is scaled) that the peak distorts the baseline.

We use the option `--remove_reference` to accomplish this. `--hlsvd` is added to make the removal more precise (at the cost of longer processing time).

In [ ]:
%sx basis_tools convert example_data/example_mrsi/3T_slaser_32vespa_1250.BASIS 3T_slaser_32vespa_1250_noTMS --remove_reference --hlsvd
%sx basis_tools conj 3T_slaser_32vespa_1250_noTMS 3T_slaser_32vespa_1250_noTMS

basis = mrs_io.read_basis('3T_slaser_32vespa_1250_noTMS')
fig = plt.figure(figsize=(8, 8))
_ = basis.plot()

## Step 3 - Remove duplicated basis spectra
No reference peaks any more! But we notice that there are two Gly basis sets. Are these actually identical?

In [ ]:
import numpy as np
np.allclose(
    basis.original_basis_array[:, basis.names.index('Gly_1')],
    basis.original_basis_array[:, basis.names.index('Gly')])

Ok, that's not good for the optimisation. We will remove the duplicate. Let's also say we aren't interested in 'HG' and we will also remove that metabolite.

In [ ]:
%sx rm 3T_slaser_32vespa_1250_noTMS/Gly_1.json
%sx rm 3T_slaser_32vespa_1250_noTMS/HG.json

basis = mrs_io.read_basis('3T_slaser_32vespa_1250_noTMS')
fig = plt.figure(figsize=(8, 8))
_ = basis.plot()
plt.xlim([5, 0])
plt.show()

## Step 4 - Add default MM peaks
The above looks good now. But you notice that there aren't any macromolecular peaks, and this is a basis set for spectra with a 32 ms echo time. We therefore need to include suitable MM peaks. Ideally this is done with a measured (metabolite nulled) MM spectrum (see the example_usage/Example basis spectra creation.ipynb notebook), but we can use the FSL-MRS 'default' MM as a (poor) alternative.

We do this by running the `basis_tools add_set` command with the `--add_MM` option.


In [ ]:
%sx basis_tools add_set --add_MM 3T_slaser_32vespa_1250_noTMS 3T_slaser_32vespa_1250_noTMS_defaultMM

basis = mrs_io.read_basis('3T_slaser_32vespa_1250_noTMS_defaultMM')
fig = plt.figure(figsize=(8, 8))
_ = basis.plot()
plt.xlim([5, 0])
plt.show()

## Conclusion
We've successfully converted an LCModel formatted basis set and added some default MM peaks. We can now use this in fitting of MRSI data!